In [ ]:
import pandas as pd
from ast import literal_eval

In [ ]:
DATA_DIR = '/home/burtenshaw/data/teen/beta/whatsapp/_6/'
ALPHA_DIR = '/home/burtenshaw/data/teen/alpha/redo/'
MATCHES_DIR = '/home/burtenshaw/data/teen/beta/combined/'

In [ ]:
# Load split data frames

user = pd.read_csv(DATA_DIR + 'wa_user_df.csv', index_col=0)
line = pd.read_csv(DATA_DIR + 'wa_line_df.csv', index_col=0)
conv = pd.read_csv(DATA_DIR + 'wa_conv_df.csv', index_col=0)

a_user = pd.read_csv(ALPHA_DIR + 'alpha_users.csv', index_col=0)
a_line = pd.read_csv(ALPHA_DIR + 'alpha_lines.csv', index_col=0)

# Encorporate validated matches
key = pd.read_csv(MATCHES_DIR + 'combined_matches.csv', index_col=0)

In [ ]:
# fields from conv
_c = conv[['line_idxs','users']]
_c['line_idxs'] = _c.line_idxs.apply(literal_eval)
_c = _c.explode('line_idxs')
_c['conv_idx'] = _c.index
_c.index = _c.line_idxs

In [ ]:
# fields from user
# LOAD ALPHA DATA - Where Possible\
_u = user[['line_idxs','AS_ALPHA_chatter_id']]
_u['chatter_id'] = key.matched_chatter_id

In [ ]:
_u = pd.merge(left=_u,right=a_user,left_on='chatter_id', right_index=True, how='left')

In [ ]:
_u['line_idxs'] = _u.line_idxs.apply(literal_eval)
_u = _u.explode('line_idxs')
_u['user_idx'] = _u.index
_u.index = _u.line_idxs

In [ ]:
_s = pd.DataFrame(list(line.source.apply(lambda x: x.split('/')[-1].split('_')[:4])),columns=['year','medium','conv_sex','conv_size'])

In [ ]:
all_on_line = pd.concat([_u,_c,_s], axis=1)
all_on_line = pd.merge(left= all_on_line, right=line, left_index=True, right_index=True)

In [ ]:
all_on_line.columns

In [ ]:
all_on_line.drop(columns=['line_idxs','line_n'], inplace=True)

In [ ]:
all_on_line.to_csv(DATA_DIR + 'all_on_line.csv')